In [122]:
from PIL import Image 
from IPython.display import display 
import random
import json

In [123]:
# Each image is made up a series of traits
# The weightings for each trait drive the rarity and add up to 100%

def create_map(list):
    result = {}
    for item in list:
        result[item] = item.lower()
    return result


background = ["Blue", "Chessboard", "Circles", "Colorsquare", "Green", "Lines", "Rainbow", "Redblue", "Waves"] 
cat = ["Black", "Blackelvis", "Brown", "Brownelvis", "Lightblack", "Lightbrown", "Tiger", "White", "Whiteelvis"] 
glass = ["Chess", "Circles", "Default", "Green", "Love", "Rainbowelvis", "Rainbow", "Starviolet", "Violet"]
hair = ["Alcohol", "Colortail", "Elvis", "Emo", "Green", "Hippy", "Rainbow", "RedBlue", "Tail"]
mustache = ["Angry", "Cute", "Default", "Holmes", "Rainbow", "Romantic", "Smile", "Volt", "Small"]

# Dictionary variable for each trait. 
# Each trait corresponds to its file name

background_files = create_map(background)
cat_files = create_map(cat)
glass_files = create_map(glass)
hair_files = create_map(hair)
mustache_files = create_map(mustache)


In [124]:
## Generate Traits


all_images = [] 

# A recursive function to generate unique image combinations
# def create_new_image():

raw_images = []
    
    
# Generate the unique combinations based on trait weightings
for bg in background:
    for c in cat:
        for g in glass:
            for h in hair:
                for m in mustache:
                    raw_images.append({
                        "Background": bg,
                        "Cat": c, 
                        "Glass": g,
                        "Hair": h,
                        "Mustache": m
                    })

for image in raw_images:
    random_image = raw_images.pop(random.randint(0, len(raw_images) - 1))
    all_images.append(random_image)


In [125]:
# Returns true if all images are unique
def all_images_unique(all_images):
    seen = list()
    return not any(i in seen or seen.append(i) for i in all_images)

# print("Are all images unique?", all_images_unique(all_images))

In [126]:
# Add token Id to each image
i = 0
for item in all_images:
    item["tokenId"] = i
    i = i + 1

In [127]:
# print(all_images)

In [128]:
# Get Trait Counts

background_count = {}
for item in background:
    background_count[item] = 0
    
cat_count = {}
for item in cat:
    cat_count[item] = 0

glass_count = {}
for item in glass:
    glass_count[item] = 0

hair_count = {}
for item in hair:
    hair_count[item] = 0

mustache_count = {}
for item in glass:
    mustache_count[item] = 0

# for image in all_images:
#     background_count[image["Background"]] += 1
#     cat_count[image["Cat"]] += 1
#     glass_count[image["Glass"]] += 1
#     hair_count[image["Hair"]] += 1
#     mustache_count[image["Mustache"]] += 1
    
print(background_count)


{'Blue': 0, 'Chessboard': 0, 'Circles': 0, 'Colorsquare': 0, 'Green': 0, 'Lines': 0, 'Rainbow': 0, 'Redblue': 0, 'Waves': 0}


In [129]:
#### Generate Metadata for all Traits 
METADATA_FILE_NAME = './metadata/all-traits.json'; 
with open(METADATA_FILE_NAME, 'w') as outfile:
    json.dump(all_images, outfile, indent=4)

In [130]:

#### Generate Images    
for item in all_images:

    background = Image.open(f'./trait-layers/backgrounds/{background_files[item["Background"]]}.png').convert('RGBA')
    cat = Image.open(f'./trait-layers/cat/{cat_files[item["Cat"]]}.png').convert('RGBA')
    hair = Image.open(f'./trait-layers/hair/{hair_files[item["Hair"]]}.png').convert('RGBA')
    mustache = Image.open(f'./trait-layers/mustache/{mustache_files[item["Mustache"]]}.png').convert('RGBA')
    glass = Image.open(f'./trait-layers/glass/{glass_files[item["Glass"]]}.png').convert('RGBA')

    #Create each composite
    com1 = Image.alpha_composite(background, cat)
    com2 = Image.alpha_composite(com1, hair)
    com3 = Image.alpha_composite(com2, mustache)
    com4 = Image.alpha_composite(com3, glass)

    #Convert to RGB
    rgb_im = com4.convert('RGB')
    file_name = str(item["tokenId"]) + ".png"
    rgb_im.save("./images/" + file_name)
    

In [ ]:
#### Generate Metadata for each Image    

f = open('./metadata/all-traits.json',) 
data = json.load(f)


IMAGES_BASE_URI = "ADD_IMAGES_BASE_URI_HERE"
PROJECT_NAME = "ADD_PROJECT_NAME_HERE"

def getAttribute(key, value):
    return {
        "trait_type": key,
        "value": value
    }
for i in data:
    token_id = i['tokenId']
    token = {
        "image": IMAGES_BASE_URI + str(token_id) + '.png',
        "tokenId": token_id,
        "name": PROJECT_NAME + ' ' + str(token_id),
        "attributes": []
    }
    token["attributes"].append(getAttribute("Background", i["Background"]))
    token["attributes"].append(getAttribute("Cat", i["Cat"]))
    token["attributes"].append(getAttribute("Glass", i["Glass"]))
    token["attributes"].append(getAttribute("Hair", i["Hair"]))
    token["attributes"].append(getAttribute("Mustache", i["Mustache"]))

    with open('./metadata/' + str(token_id), 'w') as outfile:
        json.dump(token, outfile, indent=4)
f.close()